# CLEANING DATA FOR PANDAS

## I. Column headers are values, and not variables

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# df_pew = pd.read_csv('/home/blaise/CODE/ANACONDA/DATA/pew.csv')
# df_pew.head()

<p>As it can see, the column headers are really values, which describes a "income-barcket", for which you can see in the column underneath how many ppl belongs.</p>

<em> So instead of this, we sould have on column called income, or income-bracket, and one more column with nrOfPpl value. Which makes a bit loner the whole dataset.</em>

## How to fix this? (melt())

In [ ]:
help(pd.melt)

In [ ]:
df_pew.melt(id_vars='religion')

In [ ]:
# Lets make it a bit nicer :)
df_pew_tidy = df_pew.melt(id_vars='religion',var_name='income',value_name='count')
df_pew_tidy.head()

### Second case - bilboard /melt()

In [ ]:
df_bbd = pd.read_csv('/home/blaise/CODE/ANACONDA/DATA/billboard.csv')
df_bbd.head()

In [ ]:
df_bbd.shape

In [ ]:
tidy_df_bbd = df_bbd.melt(id_vars=['year','artist','track','time','date.entered'],
    value_name='rank',var_name='week')
tidy_df_bbd.head()

In [ ]:
# Piping with dot(.) notation
# for line cntinuation use "\" character instead of beg/end parenthesis (...)
(df_bbd
     .melt(id_vars=['year','artist','track','time','date.entered'],value_name='rank',var_name='week')
     .groupby(['artist'])[['rank']]
     .agg('mean'))

## II. MULTIPLE VARIABLES IN ONE COLUMN

In [55]:
df_cs = pd.read_csv('/home/blaise/CODE/ANACONDA/DATA/country_timeseries.csv')

In [ ]:
df_cs

In [ ]:
#import matplotlib.pyplot as plt
#df_cs.plot()
#plt.show()

In [56]:
tidy_df_cs = df_cs.melt(id_vars=['Date','Day'],var_name='cd_country',value_name='count')
#tidy_df_cs.head()

In [ ]:
# Now we sould have separated country, and case/death column:
# This can be done used the string.split('_') function
# plus a string accrssor:
tidy_df_cs['cases']=tidy_df_cs['cd_country'].str.split('_',expand=True)[0]
tidy_df_cs['country']=tidy_df_cs['cd_country'].str.split('_',expand=True)[1]
tidy_df_cs.head()

In [36]:
# The 2 steps above can be one, is we save the string accessors result.
# As the result of the string manipulation (with expand=True) is a dataframe, 
# the whole dataframe can be added to the old one.
df_split =tidy_df_cs['cd_country'].str.split('_',expand=True)
df_split
tidy_df_cs[['status','country']] =df_split
tidy_df_cs.head()

,Date,Day,cd_country,count,status,country
0,1/5/2015,289,Cases_Guinea,2776.0,Cases,Guinea
1,1/4/2015,288,Cases_Guinea,2775.0,Cases,Guinea
2,1/3/2015,287,Cases_Guinea,2769.0,Cases,Guinea
3,1/2/2015,286,Cases_Guinea,NaN,Cases,Guinea
4,12/31/2014,284,Cases_Guinea,2730.0,Cases,Guinea


In [76]:
# But lets see with the correct syntax :)
tidy_df_cs['cd_country'].str.split('_').str.get(0)
tidy_df_cs['status'] = tidy_df_cs['cd_country'].str.split('_').str.get(0)
tidy_df_cs['country'] = tidy_df_cs['cd_country'].str.split('_').str.get(1)
# tidy_df_cs['status','country'] = tidy_df_cs['cd_country'].str.split('_').str[0]
# tidy_df_cs.head()

## III. Variables stored in both, rows, and columns

In [37]:
df_w = pd.read_csv('/home/blaise/CODE/ANACONDA/DATA/weather.csv')

In [38]:
df_w.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# 31 days in columns ==> melt it down to one column
df_w02 = df_w.melt(id_vars=['id','year','month','element'],var_name='day',value_name='temp')
df_w02.head()

,id,year,month,element,day,temp
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN


In [43]:
# A dataset with:
# id, year, month, minTmpr,mxTmpr, day would be nicer...
# lets see the oppsite of melt() => pivot_table()
df_w03 = df_w02.pivot_table(index=['id','year','month','day'],
                           columns=['element'],
                           values=['temp']).reset_index()
df_w03.head()

id  year month  day  temp      
element                            tmax  tmin
0        MX17004  2010     1  d30  27.8  14.5
1        MX17004  2010     2  d11  29.7  13.4
2        MX17004  2010     2   d2  27.3  14.4
3        MX17004  2010     2  d23  29.9  10.7
4        MX17004  2010     2   d3  24.1  14.4